<a href="https://colab.research.google.com/github/amanikonda123/CookbookWizard.ai/blob/main/Yet_another_copy_of_RecipeReconstruction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
import collections
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model, Input
from tensorflow.keras import backend as K
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_input_resnet
from tensorflow.keras.applications.inception_v3 import preprocess_input as preprocess_input_inception

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
od.download(
    "https://www.kaggle.com/competitions/ifood-2019-fgvc6/data")

In [ ]:
!unzip /content/ifood-2019-fgvc6/train_set.zip
!unzip /content/ifood-2019-fgvc6/test_set.zip
!unzip /content/ifood-2019-fgvc6/val_set.zip

In [ ]:
# Reading in data
df_train_labels = pd.read_csv("/content/ifood-2019-fgvc6/train_labels.csv")
df_train_labels['label'] = df_train_labels['label'].astype(str)

df_val_labels = pd.read_csv("/content/ifood-2019-fgvc6/val_labels.csv")
df_val_labels['label'] = df_val_labels['label'].astype(str)

num_classes = df_train_labels['label'].nunique()

In [ ]:
df_class_list = pd.read_csv('/content/ifood-2019-fgvc6/class_list.txt')
df_class_list.rename(columns={'0 macaron': 'Image Class'}, inplace=True)
df_class_list['Image Class'] = df_class_list['Image Class'].str.replace(r'^\d+\s+', '', regex=True)

In [ ]:
# Custom ImageDataGenerator class
class MixupImageDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, generator, dataframe, directory, batch_size, img_height, img_width, alpha=0.2, num_classes=251):
        self.batch_size = batch_size
        self.generator = generator.flow_from_dataframe(
            dataframe=dataframe,
            directory=directory,
            x_col="img_name",
            y_col="label",
            target_size=(img_height, img_width),
            batch_size=batch_size,
            class_mode="categorical")
        self.alpha = alpha
        self.num_classes = num_classes

    def __len__(self):
        return len(self.generator)

    def __getitem__(self, idx):
        x, y = next(self.generator)
        batch_size = x.shape[0]

        # Perform Mixup
        lam = np.random.beta(self.alpha, self.alpha, size=batch_size)
        index_array = np.random.permutation(batch_size)

        mixed_x = lam.reshape(batch_size, 1, 1, 1) * x + (1 - lam).reshape(batch_size, 1, 1, 1) * x[index_array]
        mixed_y = lam.reshape(batch_size, 1) * y + (1 - lam).reshape(batch_size, 1) * y[index_array]

        return mixed_x, mixed_y

    def on_epoch_end(self):
        self.generator.on_epoch_end()

In [ ]:
# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_mixup_generator = MixupImageDataGenerator(
    generator=train_datagen,
    dataframe=df_train_labels,
    directory='/content/train_set',
    batch_size=64,
    img_height=224,
    img_width=224,
    alpha=0.2,
)

val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=df_val_labels,
    directory='/content/val_set',
    x_col='img_name',
    y_col='label',
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
# Callbacks - ResNet50
callbacks_resnet = [
    ModelCheckpoint('best_model_resnet.h5', monitor='val_accuracy', save_best_only=True, mode='max'),
    EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.00001)
]

In [ ]:
# Base model - ResNet50
base_model_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x_resnet = base_model_resnet.output
x_resnet = GlobalAveragePooling2D()(x_resnet)
x_resnet = Dense(1024, activation='relu')(x_resnet)
predictions_resnet = Dense(num_classes, activation='softmax')(x_resnet)
model_resnet = Model(inputs=base_model_resnet.input, outputs=predictions_resnet)

model_resnet.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Training - ResNet50
history_resnet = model_resnet.fit(
    train_mixup_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=callbacks_resnet
)

In [ ]:
# Callbacks - InceptionV3
callbacks_inception = [
    ModelCheckpoint('best_model_inception.h5', monitor='val_accuracy', save_best_only=True, mode='max'),
    EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.00001)
]

In [ ]:
# Base model - InceptionV3
base_model_inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x_inception = base_model_inception.output
x_inception = GlobalAveragePooling2D()(x_inception)
x_inception = Dense(1024, activation='relu')(x_inception)
predictions_inception = Dense(num_classes, activation='softmax')(x_inception)
model_inception = Model(inputs=base_model_inception.input, outputs=predictions_inception)

model_inception.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Training - InceptionV3
history_inception = model_inception.fit(
    train_mixup_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=callbacks_inception
)

In [ ]:
# Ensembling the two models
def ensemble_predictions(models, generator, weights):
    """Generate ensemble predictions from multiple models."""
    total_predictions = None
    for model, weight in zip(models, weights):
        generator.reset()
        predictions = model.predict(generator)
        if total_predictions is None:
            total_predictions = predictions * weight
        else:
            total_predictions += predictions * weight

    return total_predictions

models = [model_resnet, model_inception]
weights = [0.6, 0.4]

ensemble_pred = ensemble_predictions(models, val_generator, weights)

predicted_classes = np.argmax(ensemble_pred, axis=1)

accuracy = accuracy_score(val_generator.classes, predicted_classes)
print(f"Ensemble Validation Accuracy: {accuracy*100:.2f}%")

report = classification_report(val_generator.classes, predicted_classes, target_names=list(val_generator.class_indices.keys()))
print("\nClassification Report:")
print(report)


In [ ]:
# Plot the Confusion Matrix
cm = confusion_matrix(val_generator.classes, predicted_classes)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap='Blues', xticklabels=list(val_generator.class_indices.keys()), yticklabels=list(val_generator.class_indices.keys()))
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()


In [ ]:
# Preprocess input image, load the models, and predict the class
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array_resnet = preprocess_input_resnet(img_array.copy())
    img_array_inception = preprocess_input_inception(img_array.copy())
    return img_array_resnet, img_array_inception

model_resnet = load_model('best_model_resnet.h5')
model_inception = load_model('best_model_inception.h5')

def predict_image_class(image_path, models, weights):
    img_array_resnet, img_array_inception = load_and_preprocess_image(image_path)
    img_array_resnet = np.expand_dims(img_array_resnet, axis=0)
    img_array_inception = np.expand_dims(img_array_inception, axis=0)

    predictions_resnet = models[0].predict(img_array_resnet)
    predictions_inception = models[1].predict(img_array_inception)

    ensemble_prediction = predictions_resnet * weights[0] + predictions_inception * weights[1]
    class_index = np.argmax(ensemble_prediction, axis=1)
    return class_index

In [ ]:
# Upload Image from Google Drive Here
image_path = 'uploaded_image.jpg'

models = [model_resnet, model_inception]
weights = [0.6, 0.4]

predicted_class_index = predict_image_class(image_path, models, weights)
print(f"The image is classified as: {df_class_list.iloc[predicted_class_index, 0]}")